In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
from code.python.mpmath_integration import quad_phi
from mpmath import mp

from scipy import stats

In [3]:
x = mp.mpf('2/10')
alpha = mp.mpf('1')
beta = mp.mpf('1/2')
mu = mp.mpf('1/10')
delta = mp.mpf('1/5')

xmu = x - mu
gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

In [4]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.60151937190920007')

In [5]:
cache = {}

def Ik(k):
    s = 0
    for j in range(2*k + 1 + 1):
        m = k + 1 - j
        if abs(m) not in cache:
            bm = mp.besselk(m, alpha * omega)
            cache[m] = bm
        else:
            bm = cache[abs(m)]
            
        s += mp.binomial(2*k + 1, j) * (-1) ** (-j) * (omega * beta / alpha / xmu) ** (j) * bm

    return 2 * s

In [6]:
cache

{}

In [7]:
%%time
mp.dps = 100

C = delta * mp.exp(delta * gamma + xmu * beta) / (2 * mp.pi)

N = 100
s = 0

z = xmu ** 2 * alpha / omega

for k in range(N):
    r = z ** k / mp.fac2(2*k + 1)
    # q = mp.quad(lambda t: (t - xmu / beta)**(2*k + 1) * t**(-k - 2) * mp.exp(-omega ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    q = Ik(k)
    t = r * q
    s += t
    print(k, float(r), float(q), float(t))

mp.mpf('1/2') + C * s * xmu * alpha / omega, s

0 1.0 4.785709029166753 4.785709029166753
1 0.014907119849998597 51.14411800466839 0.762411496718063
2 0.00013333333333333334 968.3197452700285 0.12910929936933713
3 8.5183541999992e-07 26614.769317939328 0.02267140320014783
4 4.232804232804233e-09 962477.2038680709 0.004073977582510353
5 1.7208796363634747e-11 43204352.04187382 0.0007434948963113936
6 5.92000592000592e-14 2316732137.901176 0.00013715067971442933
7 1.7650047552445893e-16 144426055009.59885 2.549126738731586e-05
8 4.64314189804386e-19 10257411172889.979 4.762661558230866e-06
9 1.0928822013898386e-21 816970165661910.6 8.9285215311841e-07
10 2.3273894225783755e-24 7.205002666454633e+16 1.6768846995555504e-07
11 4.5253921382602014e-27 6.961990379738547e+18 3.150573653111198e-08
12 8.09526755679435e-30 7.304113654777799e+20 5.912875430066133e-09
13 1.3408569298548746e-32 8.253617130438279e+22 1.106691972571707e-09
14 2.0677567194877013e-35 9.971093318527516e+24 2.0617795210024192e-10
15 2.982996506907396e-38 1.2786303760617

(mpf('0.6015193719092000535521303489312528824733486539076138586577812188441866608955752432655167301598077635432'),
 mpf('5.70488720486054057304110404906455351223687778650824486241975007185268495951987606124719168848771679489'))

Series with Remainder and integral

In [8]:
mp.dps = 100

C = delta * mp.exp(delta * gamma + xmu * beta) / (2 * mp.pi)

s = 0

z = xmu ** 2 * alpha / omega

for k in range(N):
    r = (-beta) ** (2*k+1) / mp.fac2(2*k + 1)
    q = mp.quad(lambda t: (t - xmu / beta)**(2*k + 1) * t**(-k - 2) * mp.exp(-omega ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    t = r * q
    s += t
    print(k, float(r), float(q), float(t))

float(s)

0 -0.5 -4.280468283900554 2.140234141950277
1 -0.041666666666666664 -8.183058880746943 0.3409607866977893
2 -0.0020833333333333333 -27.71492831205202 0.05773943398344171
3 -7.440476190476191e-05 -136.26761890784937 0.010138959740167363
4 -2.066798941798942e-06 -881.5265606217441 0.0018219381626606815
5 -4.697270322270322e-08 -7078.601038540606 0.0003325010258152867
6 -9.033212158212158e-10 -67900.15282060258 6.133564860035308e-05
7 -1.5055353597020263e-11 -757208.4752887256 1.1400041342132343e-05
8 -2.2140225877970977e-13 -9620168.336787779 2.1299269996058576e-06
9 -2.913187615522497e-15 -137064849.3486566 3.9929562164596313e-07
10 -3.468080494669639e-17 -2162365137.947989 7.499256357271044e-08
11 -3.769652711597434e-19 -37376901245.0546 1.408979371295296e-08
12 -3.7696527115974335e-21 -701475303729.6836 2.6443182808232354e-09
13 -3.490419177405031e-23 -14179606259575.111 4.949276961647339e-10
14 -3.008982049487096e-25 -306434474367450.75 9.22055832715673e-11
15 -2.4265984270057226e-27

2.5513031188073874

In [9]:
remainder = 0
for k in range(N, N + 50):
    r = (-beta) ** (2*k+1) / mp.fac2(2*k + 1)
    q = mp.quad(lambda t: (t - xmu / beta)**(2*k + 1) * t**(-k - 2) * mp.exp(-omega ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    t = r * q
    remainder += t
    # print(k, float(r), float(q), float(t))

float(remainder)

-2.9752590863204527e-64

In [10]:
def fremainder(t):
    t1 = t **(-3/2) * mp.exp(-delta**2 / 2 / t - gamma ** 2 / 2 * t - beta * xmu)
    t2 = mp.gammainc(N + 1/2 , 0, (beta * t - xmu) ** 2 / 2 / t, regularized=True)
    return t1 * t2


def fremainder_approx(t):
    t1 = t **(-3/2) * mp.exp(-delta**2 / 2 / t - gamma ** 2 / 2 * t - beta * xmu)
    z = (beta * t - xmu) ** 2 / 2 / t
    a = N + 1/2
    t2 = z ** a* mp.exp(-z) / a / mp.gamma(N + 1/2)
    return t1 * t2


def fremainder_approx2(t):
    t1 = t **(-3/2) * mp.exp(-delta**2 / 2 / t - gamma ** 2 / 2 * t - (beta * t - xmu) ** 2 / 2 / t)
    t2 = ((beta * t - xmu) / mp.sqrt(t))**(2*N + 1) / mp.gamma(N + 1/2)
    return t1 * t2 * mp.exp(- beta * xmu)

In [11]:
float(mp.quad(fremainder, [0, mp.inf]))

2.3739139831537993e-64

In [12]:
float(mp.quad(fremainder_approx, [0, mp.inf]))

1.789160643466692e-64

In [13]:
float(mp.quad(fremainder_approx2, [0, mp.inf]))

3.2235159796048685e-32

In [14]:
remainder = (-1) ** (2*N + 1) * 2 ** (N - 1/2) * (2*N + 1) * mp.gamma(N + 1/2) / mp.fac2(2*N + 1) * mp.quad(fremainder, [0, mp.inf])
float(remainder)

-2.9752599558576072e-64

Approximation 1

In [15]:
def fremainder_approx(t):
    t1 = t **(-3/2) * mp.exp(-delta**2 / 2 / t - gamma ** 2 / 2 * t - beta * xmu)
    z = (beta * t - xmu) ** 2 / 2 / t
    a = N + 1/2
    t2 = z ** a* mp.exp(-z) / a / mp.gamma(N + 1/2)
    return t1 * t2

In [16]:
remainder = (-1) ** (2*N + 1) * 2 ** (N - 1/2) * (2*N + 1) * mp.gamma(N + 1/2) / mp.fac2(2*N + 1) * mp.quad(fremainder_approx, [0, mp.inf])
float(remainder)

-2.2423803283853027e-64

Approximation 2

In [17]:
def fremainder_approx2(t):
    t1 = t **(-3/2) * mp.exp(-delta**2 / 2 / t - gamma ** 2 / 2 * t - (beta * t - xmu) ** 2 / 2 / t)
    t2 = ((beta * t - xmu) / mp.sqrt(t))**(2*N + 1)
    return t1 * t2 * mp.exp(- beta * xmu) 

In [18]:
 mp.quad(fremainder_approx2, [0, mp.inf])

mpf('3.004627351131803554470967013793726883419408820197767040162778468103911873729605466090818036508118293746e+125')

In [19]:
remainder_approx2 = (-1) ** (2*N + 1) / mp.fac2(2*N + 1) * mp.quad(fremainder_approx2, [0, mp.inf])
float(remainder_approx2)

-2.242379631903045e-64

Naive

In [20]:
def fremainder_approx_naive(t):
    t1 = t **(-2 - N) * mp.exp(-omega**2 / 2 / t - alpha ** 2 / 2 * t)
    t2 = ((-xmu))**(2*N + 1)
    return t1 * t2

In [21]:
remainder_naive = (-1) ** (2*N + 1) / mp.fac2(2*N + 1) * mp.quad(fremainder_approx_naive, [0, mp.inf])
float(remainder_naive)

4.47638071148356e-71

In [22]:
mp.quad(fremainder_approx_naive, [0, mp.inf])

mpf('-5.998028045049549276414816139280174966310572753523996177430158620655292544153922614492753754405410589684e+118')

In [23]:
float((-1) ** (2*N + 1) / mp.fac2(2*N + 1) * 2 * ((-xmu))**(2*N + 1) * mp.besselk(N+1 , omega * alpha) * (omega / alpha) **(-N-1))

4.47638071148356e-71

Approximation 3

In [24]:
def fremainder_approx3(t):
    t1 =  mp.exp(-omega**2 / 2 / t - alpha ** 2 / 2 * t)
    t2 = t**(N -1)
    return t1 * t2

In [25]:
float(mp.quad(fremainder_approx3, [0, mp.inf]))

1.182900964729811e+186

In [26]:
float(beta **(2*N + 1) * (mp.quad(fremainder_approx3, [0, mp.inf])))

3.680605387855149e+125

In [27]:
beta ** (2*N + 1) * mp.quad(fremainder_approx3, [0, mp.inf])

mpf('3.680605387855149368871667131265444404438110462713173387416148964486159852504562278207952240821927162942e+125')

In [28]:
float(beta ** (2*N + 1) * 2 * mp.besselk(N , omega * alpha) * (omega / alpha) **(N) / mp.fac2(2*N + 1))

2.7468679441029777e-64

In [29]:
float(beta ** (2*N + 1) * mp.sqrt(mp.pi * 2/  N) * (mp.exp(1) * alpha ** 2/ 2 / N) ** (-N)  / mp.fac2(2*N + 1))

2.744926408129935e-64